In [ ]:
# !pip install sqlite3

In [ ]:
import pandas as pd
from tqdm import tqdm
import sqlite3

# И задаем запрос

In [ ]:
db_file = 'sem_notes_corp.db'

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Запрос с несколькими JOIN
cursor.execute("""
    SELECT DISTINCT doc_name
    FROM DOCS
""")

# Получение результата
res = cursor.fetchall()

# Закрываем подключение
conn.close()

doc_names = [r[0] for r in res]
doc_names

['История семантики: основные этапы и имена.',
 'Треугольник Фреге.',
 'Теория референции.',
 'Понятие интенсионала и экстенсионала.',
 'Компонентный анализ.',
 'Фон и фигура.',
 'Траектор и ориентир.',
 'Семантические роли.',
 'Фреймовая семантика.',
 'Синкретическое выражение семантических ролей.',
 'Актанты и сирконстанты.',
 'Семантические и синтаксические актанты.',
 'Наследование актантов.',
 'Компоненты значения.',
 'Понятие сферы действия.',
 'Фигура наблюдателя.',
 'Понятие коннотации.',
 'Отрицательная и положительная поляризация.',
 'Полисемия.',
 'Понятие семантического инварианта.',
 'Основные типы семантических сдвигов.',
 'Р.Якобсон о метафоре и метонимии.',
 'А.Вежбицка о метафоре и сравнении.',
 'Дж.Лакофф о концептуальной метафоре.',
 'Концептуальный блендинг.',
 'Е.В.Падучева о механизмах метафоры и метонимии.',
 'Регулярная метонимия (типы с примерами).',
 'Универсальная метафора.',
 'Лексические функции.',
 'Импликатура как механизм семантических сдвигов.',
 'Грамм

In [33]:
import sqlite3

def get_sentences_with_lemma_only(db_file, lemma, doc_name=None, gender=None):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданной леммой.
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    # Запрос с несколькими JOIN
    if (doc_name != None) and (gender != None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.lemma = ? AND DOCS.gender = ? AND DOCS.doc_name IN ({seq})
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.execute(query, tuple([lemma] + [gender] + doc_name))
    elif (doc_name == None) and (gender != None):
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.lemma = ? AND DOCS.gender = ?
        """, (lemma, gender))
    elif (doc_name != None) and (gender == None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.lemma = ? AND DOCS.doc_name IN ({seq})
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.execute(query, tuple([lemma] + doc_name))
    else:
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.lemma = ?
        """, (lemma,))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [43]:
def get_sentences_with_wordform_only(db_file, wordform, doc_name=None, gender=None):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданной словоформой.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    # Запрос с несколькими JOIN
    if (doc_name != None) and (gender != None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.word = ? AND DOCS.doc_name IN ({seq}) AND DOCS.gender = ?
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.execute(query, tuple([wordform] + doc_name + [gender]))
    elif (doc_name == None) and (gender != None):
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.word = ? AND DOCS.gender = ?
        """, (wordform, gender))
    elif (doc_name != None) and (gender == None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.word = ? AND DOCS.doc_name IN ({seq})
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.execute(query, tuple([wordform] + doc_name))
    else:
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.word = ?
        """, (wordform,))


    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [45]:
def get_sentences_with_lemma_tag(db_file, lemma, tag, doc_name=None, gender=None):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданной леммой и тегом.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    if (doc_name != None) and (gender != None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.lemma = ? and WORD.tag = ? AND DOCS.doc_name IN ({seq}) AND DOCS.gender = ?
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.exectue(query, tuple([lemma] + [tag] + doc_name + [gender]))
    elif (doc_name == None) and (gender != None):
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.lemma = ? and WORD.tag = ? AND DOCS.gender = ?
        """, (lemma, tag, gender))
    elif (doc_name != None) and (gender == None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.lemma = ? and WORD.tag = ? AND DOCS.doc_name IN ({seq})
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.execute(query, tuple([lemma] + [tag] + doc_name))
    else:
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.lemma = ? and WORD.tag = ?
        """, (lemma, tag))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [47]:
def get_sentences_with_wordform_tag(db_file, wordform, tag, doc_name=None, gender=None):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданной словоформой и тегом.
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    if (doc_name != None) and (gender != None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.word = ? and WORD.tag = ? AND DOCS.doc_name IN ({seq}) AND DOCS.gender = ?
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.execute(query, tuple([wordform] + [tag] + doc_name + [gender]))
    elif (doc_name == None) and (gender != None):
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.word = ? and WORD.tag = ? AND DOCS.gender = ?
        """, (wordform, tag, gender))
    elif (doc_name != None) and (gender == None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.word = ? and WORD.tag = ? AND DOCS.doc_name IN ({seq})
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.execute(query, tuple([wordform] + [tag] + doc_name))
    else:
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.word = ? and WORD.tag = ?
        """, (wordform, tag))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [51]:
def get_sentences_with_tag(db_file, tag, doc_name=None, gender=None):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданным тегом.
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    if (doc_name != None) and (gender != None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.tag = ? AND DOCS.doc_name IN ({seq}) AND DOCS.gender = ?
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.execute(query, tuple([tag] + doc_name + [gender]))
    elif (doc_name == None) and (gender != None):
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.tag = ? AND DOCS.gender = ?
        """, (tag, gender))
    elif (doc_name != None) and (gender == None):
        query = """
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.tag = ? AND DOCS.doc_name IN ({seq})
        """.format(seq=','.join(['?']*len(doc_name)))
        cursor.execute(query, tuple([tag] + doc_name))
    else:
        cursor.execute("""
            SELECT DISTINCT WORD.sent_id, WORD.word_position, DOCS.doc_id
            FROM WORD
            JOIN SENT ON SENT.sent_id = WORD.sent_id
            JOIN DOCS on DOCS.doc_id = SENT.doc
            WHERE WORD.tag = ?
        """, (tag, ))


    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [53]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=644887a09b669d85f19219d367019848e6a8a52ad5b475fb49dbbcea70380179
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [54]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [55]:
all_tags = ['A', 'ADV', 'ADVPRO', 'ANUM', 'APRO',
            'COM', 'CONJ', 'INTJ', 'NUM', 'PART',
            'PR', 'S', 'SPRO', 'V']

In [56]:
def search(query, db_file, doc_name=None, gender=None):
    """Выбирает предложения из таблицы SENT,
    которые содержат поисковый запрос.
    """

    # делим запрос на части
    query_parts = query.split()
    # счетчик по предложениям
    wp = None
    # для каждой части запроса
    for query_part in query_parts:
        # делим запрос по знаку + (если вдруг есть тег)
        part = query_part.split('+')
        # если есть тег
        if len(part) == 2:
            # отдельно записываем слово и тег
            word, tag = part[0], part[1]
            # если мы ищем словоформу
            if word[0] and word[-1] == '"':
                # находим все предложения со словоформой
                # (предложение, номер в предложении)
                sentences = get_sentences_with_wordform_tag(db_file, word, tag, doc_name, gender)
                # если счетчик не задан (первый элемент запроса)
                if wp == None:
                    # делаем словарь
                    # предложение: номер в предложении
                    wp = {s[0]: [s[1]] for s in sentences}
                # если уже задан (то есть не первый элемент)
                else:
                    for s in sentences:
                        if s[0] in wp:
                            if wp[s[0]][-1] == s[1] - 1:
                                wp[s[0]].append(s[1])
                            else:
                                del wp[s[0]]
            # если ищем лемму
            else:
                # собственно лемма
                word = morph.parse(word)[0].normal_form
                # дальше то же самое
                sentences = get_sentences_with_lemma_tag(db_file, word, tag, doc_name, gender)
                if wp == None:
                    wp = {s[0]: [s[1]] for s in sentences}
                    # sents.append(sentences)
                else:
                    for s in sentences:
                        if s[0] in wp:
                            if wp[s[0]][-1] == s[1] - 1:
                                wp[s[0]].append(s[1])
                            else:
                                del wp[s[0]]
        # если тега нет
        else:
            # то же самое, что раньше
            word = part[0]
            if word[0] and word[-1] == '"':
                sentences = get_sentences_with_wordform_only(db_file, word, doc_name, gender)
                if wp == None:
                    wp = {s[0]: [s[1]] for s in sentences}
                    # sents.append(sentences)
                else:
                    for s in sentences:
                        if s[0] in wp:
                            if wp[s[0]][-1] == s[1] - 1:
                                wp[s[0]].append(s[1])
                            else:
                                del wp[s[0]]
            else:
                if word not in all_tags:
                    word = morph.parse(word)[0].normal_form
                    sentences = get_sentences_with_lemma_only(db_file, word, doc_name, gender)
                else:
                    sentences = get_sentences_with_tag(db_file, word, doc_name, gender)
                if wp == None:
                    wp = {s[0]: [s[1]] for s in sentences}
                    # sents.append(sentences)
                else:
                    for s in sentences:
                        if s[0] in wp:
                            if wp[s[0]][-1] == s[1] - 1:
                                wp[s[0]].append(s[1])
                            else:
                                del wp[s[0]]
    sents = [elem for elem in wp if len(wp[elem]) == len(query_parts)]
    # sents = list(wp.keys())
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    # Запрос с несколькими JOIN
    cursor.execute("""
        SELECT DISTINCT DOCS.gender, DOCS.doc_name, SENT.sentence_text
        FROM SENT
        JOIN DOCS on DOCS.doc_id = SENT.doc
        WHERE SENT.sent_id IN ({seq})""".format(
    seq=','.join(['?']*len(sents))), sents)

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    output = {}

    for r in result:
        if r[2] not in output:
            output[r[2]] = {'Название конспекта': r[1],
                            'Пол автора': r[0],
                            'Год': 2023}
    return output

In [59]:
%time
search('A рамка', 'sem_notes_corp.db', doc_name=['Компоненты значения.', 'Отрицательная и положительная поляризация.'])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


{'Модальная рамка.': {'Название конспекта': 'Компоненты значения.',
  'Пол автора': 'f',
  'Год': 2023},
 'Это не пресуппозиция и не ассерция: сочетание отрицания с, например, словами, обеспечивающими модальную рамку, в принципе аномально (см.': {'Название конспекта': 'Компоненты значения.',
  'Пол автора': 'f',
  'Год': 2023},
 'Заметим, что *Он выпил не всего два бокала вина нельзя, слова с модальной рамкой с отрицанием не сочетаются.': {'Название конспекта': 'Компоненты значения.',
  'Пол автора': 'f',
  'Год': 2023},
 'PPI не сочетаются с отрицанием, потому что многие PPI – слова, состоящие из одной модальной рамки.': {'Название конспекта': 'Отрицательная и положительная поляризация.',
  'Пол автора': 'f',
  'Год': 2023}}